# Demo Notebook - CodeSystem

In [34]:
PHENOP="http://localhost:8088/fhir" # Jupyter uses 8888!!

import httpx
import json
from IPython.display import display, HTML

client = httpx.Client()
request_headers = {
    "Content-Type" : "application/fhir+json; charset=UTF-8",
    "Accept-Charset" : "utf-8",
    "User-Agent" : "HAPI-FHIR/5.0.0 (FHIR Client; FHIR 4.0.1/R4; apache)" }

def do_hapi_request(internal_path, in_params):
    rq = httpx.Request("GET", PHENOP + internal_path,
               params = in_params,  headers=request_headers)
    print(rq.url)
    response = client.send(rq)
    return response
    
def decode_status(response):
    if response is None:
        print("NO RESPONSE")
        return(True)
    else:
        if (response.status_code >= 200 and response.status_code < 300):
            return(False)
        elif (response.status_code >= 400 and response.status_code < 500):
            return(True)
        elif (response.status_code >= 500) :
            return(True)

def decode_code_system_json(response):
    if not decode_status(response) :
        response_dict = json.loads(response.text)
        if ('entry' in response_dict):
            print("title:" + response_dict['entry'][0]['resource']['title'])
            print("resourceType:", response_dict['entry'][0]['resource']['resourceType'],
                 end="  ")
            print("id: " + response_dict['entry'][0]['resource']['id'])
            print("url: " + response_dict['entry'][0]['resource']['url'])
            print("full url: " + response_dict['entry'][0]['fullUrl'])
            if len(response_dict['entry']) > 1 :
                print("identifier system:\"" + response_dict['entry'][1]['resource']['identifier'][0]['system'],
                      end="\",  ")
                print("value: \"" + response_dict['entry'][1]['resource']['identifier'][0]['value'] + "\"")
            else:
                print("?? no 2nd entry for resource identifier")
        else:
            print("Not Present")
        print("")  

def decode_validate_response(repsonse) :
    print(response)
    if False:
        print(response.keys())
        print(response['issue']['severity'])
        print(response['issue']['response'])
        display(HTML(response['text']['div']))
        print("")
        
def decode_lookup_response(response):
    if (not decode_status(response)) :
        response_dict = json.loads(response.text)
        for p in response_dict['parameter']:
            if p['name'] != 'property' and p['name'] != 'designation':
                if 'valueString' in p:
                    print(p['name'], ":", p['valueString'], sep="", end=", ")
                if 'valueBoolean' in p:
                    print(p['name'], ":", p['valueBoolean'], sep="", end=", ")
        print("")
    else:
        print("error")
    print("")

## CodeSystem Search
List code systems 
(where's Loinc?)

In [41]:
response = do_hapi_request("/CodeSystem/", {})    
if (not decode_status(response)) :
    response_dict = json.loads(response.text)
    for vocab_entry in response_dict['entry']:
        print(vocab_entry['resource']['title'])
        print(vocab_entry['resource']['id'])
        print(vocab_entry['fullUrl'])
        print(vocab_entry['resource']['url'])
        print("")

http://localhost:8088/fhir/CodeSystem/
American Dental Association Area of Oral Cavity System
ADAAreaOralCavitySystem
http://localhost:8088/fhir/CodeSystem/ADAAreaOralCavitySystem
http://terminology.hl7.org/CodeSystem/ADAAreaOralCavitySystem

ADA Universal Tooth Designation System
ADAUniversalToothDesignationSystem
http://localhost:8088/fhir/CodeSystem/ADAUniversalToothDesignationSystem
http://terminology.hl7.org/CodeSystem/ADAUniversalToothDesignationSystem

AHA NUBC Patient Discharge Status Codes
AHANUBCPatientDischargeStatus
http://localhost:8088/fhir/CodeSystem/AHANUBCPatientDischargeStatus
https://www.nubc.org/CodeSystem/PatDischargeStatus

AHA NUBC Point of Origin for Newborn
AHANUBCPointOfOriginNewborn
http://localhost:8088/fhir/CodeSystem/AHANUBCPointOfOriginNewborn
https://www.nubc.org/CodeSystem/PointOfOriginNewborn

AHA NUBC Point of Origin for Non-newborn
AHANUBCPointOfOriginNonnewborn
http://localhost:8088/fhir/CodeSystem/AHANUBCPointOfOriginNonnewborn
https://www.nubc.org

## CodeSystem search 
https://hl7.org/fhir/http.html#search 

In [21]:
response = do_hapi_request("/CodeSystem/", {
    'system' : 'http://loinc.org'})    
if (not decode_status(response)) :
    decode_code_system_json(response)

http://localhost:8088/fhir/CodeSystem/?system=http%3A%2F%2Floinc.org
title:Logical Observation Identifiers, Names and Codes (LOINC)
resourceType: CodeSystem  id: v3-loinc
url: http://loinc.org
full url: http://localhost:8088/fhir/CodeSystem/v3-loinc
identifier system:"urn:ietf:rfc:3986",  value: "urn:oid:2.16.840.1.113883.6.1"



In [22]:
# fails
response = do_hapi_request("/CodeSystem/", {
    'system' : 'v3-loinc'})
if (not decode_status(response)) :
    decode_code_system_json(response)
    print(response)

http://localhost:8088/fhir/CodeSystem/?system=v3-loinc
Not Present

<Response [200 OK]>


In [24]:
response = do_hapi_request("/CodeSystem/", {'system' : 'http://snomed.org'}) 
decode_code_system_json(response)

http://localhost:8088/fhir/CodeSystem/?system=http%3A%2F%2Fsnomed.org
Not Present



In [25]:
response = do_hapi_request("/CodeSystem/", {'system' : 'http://www.ama-assn.org/go/cpt'}) 
decode_code_system_json(response)

http://localhost:8088/fhir/CodeSystem/?system=http%3A%2F%2Fwww.ama-assn.org%2Fgo%2Fcpt
title:Current Procedural Terminology (CPT®)
resourceType: CodeSystem  id: CPT
url: http://www.ama-assn.org/go/cpt
full url: http://localhost:8088/fhir/CodeSystem/CPT
?? no 2nd entry for resource identifier



In [26]:
response = do_hapi_request("/CodeSystem/", {'system' : 'http://hpo.org'}) 
decode_code_system_json(response)

http://localhost:8088/fhir/CodeSystem/?system=http%3A%2F%2Fhpo.org
Not Present



In [27]:
response = do_hapi_request("/CodeSystem/", {'system' : 'http://ICD10CM.org'}) 
decode_code_system_json(response)

http://localhost:8088/fhir/CodeSystem/?system=http%3A%2F%2FICD10CM.org
Not Present



## CodeSystem $validate-code 

In [42]:
response = do_hapi_request("/CodeSystem/v3-loinc/$validate-code", {
    'code' : '69551-0'})
if (not decode_status(response)) :
    response_dict = json.loads(response.text)
    decode_validate_response(response_dict)

http://localhost:8088/fhir/CodeSystem/v3-loinc/$validate-code?code=69551-0
<Response [200 OK]>


In [58]:
response = do_hapi_request("/CodeSystem/v3-loinc/$validate-code", {
    'codeSystem' : 'http://loinc.org',
    'code' : '69551-0'})
if (not decode_status(response)) :
    response_dict = json.loads(response.text)
    decode_validate_response(response_dict)
else:
    print(response, end="\n\n")

http://localhost:8088/fhir/CodeSystem/v3-loinc/$validate-code?codeSystem=http%3A%2F%2Floinc.org&code=69551-0
<Response [200 OK]>


In [49]:
response = do_hapi_request("/CodeSystem/v3-loinc/$validate-code", {
    'code' : 'LA6668-3'})
if (not decode_status(response)) :
    response_dict = json.loads(response.text)
    decode_validate_response(response_dict)
else:
    print(response, end="\n\n")

http://localhost:8088/fhir/CodeSystem/v3-loinc/$validate-code?code=LA6668-3
<Response [200 OK]>


In [50]:
response = do_hapi_request("/CodeSystem/$validate-code/", {
    'codeSystem' : 'http://loinc.org',
    'code' : '69551-0'})
if (not decode_status(response)) :
    response_dict = json.loads(response.text)
    decode_validate_response(response_dict)
else:
    print(response, end="\n\n")

http://localhost:8088/fhir/CodeSystem/$validate-code/?codeSystem=http%3A%2F%2Floinc.org&code=69551-0
<Response [400 Bad Request]>



In [51]:
# with codeSystem parameter only, fails
response = do_hapi_request("/CodeSystem/$validate-code/", {
    'codeSystem' : 'v3-loinc',
    'code' : '69551-0'}) 
if (not decode_status(response)) :
    response_dict = json.loads(response.text)
    decode_validate_response(response_dict)
else:
    print(response, end="\n\n")

http://localhost:8088/fhir/CodeSystem/$validate-code/?codeSystem=v3-loinc&code=69551-0
<Response [400 Bad Request]>



## CodeSystem $lookup 

In [52]:
response = do_hapi_request("/CodeSystem/$lookup", {
    'system' : 'http://bogus.org',
    'code' : 'LA6668-3'})   
decode_lookup_response(response)

http://localhost:8088/fhir/CodeSystem/$lookup?system=http%3A%2F%2Fbogus.org&code=LA6668-3
error



In [53]:
response = do_hapi_request("/CodeSystem/$lookup", {
    'system' : 'http://loinc.org',
    'code' : 'bogus'})   
decode_lookup_response(response)

http://localhost:8088/fhir/CodeSystem/$lookup?system=http%3A%2F%2Floinc.org&code=bogus
error



In [54]:
response = do_hapi_request("/CodeSystem/$lookup", {
    'system' : 'http://loinc.org',
    'code' : 'LA6668-3'})   
decode_lookup_response(response)

http://localhost:8088/fhir/CodeSystem/$lookup?system=http%3A%2F%2Floinc.org&code=LA6668-3
name:LOINC, display:Pathogenic, abstract:False, 



In [55]:
response = do_hapi_request("/CodeSystem/$lookup", {
    'system' : 'http://loinc.org',
    'code' : 'LA9633-4'}) 
decode_lookup_response(response)

http://localhost:8088/fhir/CodeSystem/$lookup?system=http%3A%2F%2Floinc.org&code=LA9633-4
name:LOINC, display:Present, abstract:False, 



In [56]:
response = do_hapi_request("/CodeSystem/$lookup", {
    'system' : 'http://loinc.org',
    'code' : '69547-8'})
decode_lookup_response(response)

http://localhost:8088/fhir/CodeSystem/$lookup?system=http%3A%2F%2Floinc.org&code=69547-8
name:LOINC, display:Genomic ref allele [ID], abstract:False, 



In [57]:
response = do_hapi_request("/CodeSystem/$lookup", {
    'system' : 'http://loinc.org',
    'code' : '69551-0'}) 
decode_lookup_response(response)

http://localhost:8088/fhir/CodeSystem/$lookup?system=http%3A%2F%2Floinc.org&code=69551-0
name:LOINC, display:Genomic alt allele [ID], abstract:False, 



## CodeSystem $subsumes 

In [ ]:

# T B D


# What else?
- search for concept by name, or pattern
- modify concept? (no)
- hierarchy closure, synonym closure
- more